In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

import matplotlib.pyplot as plt

from scipy import stats
import math
import random

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error

from xgboost import XGBRegressor

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_test['SalePrice'] = 0
df_concat = pd.concat([df_train,df_test])

In [3]:
def giveMeWrangledData(df, testFile=False, log=False):
    
    
    df = df.drop(['Id', 'GarageYrBlt','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF'],axis=1)
    
    df['LotFrontage'] =df.LotFrontage.fillna(df.LotFrontage.mode()[0])
    df['MasVnrArea']=df.MasVnrArea.fillna(0.0)
    df['TotalBsmtSF'] = df.TotalBsmtSF.fillna(0)
    df['BsmtFullBath'] = df.BsmtFullBath.fillna(0)
    df['BsmtHalfBath'] = df.BsmtHalfBath.fillna(0)
    df['GarageCars'] = df.GarageCars.fillna(0)
    df['GarageArea'] = df.GarageArea.fillna(0)
    
    #convert data type
    #we are being little lineant to give int64 for YearBuilt, YrSold but those guys are going to be box-coxed 
    #so let them at least enjoy the bigger size for now
    int64_variables = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', \
                     'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', \
                     'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',\
                     'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', \
                     'PoolArea', 'MiscVal', 'YrSold', 'SalePrice']
    
    #if testFile:
    #    int64_variables.remove('SalePrice')

        
        
        
    
    for c in int64_variables:
        if log:
            print("Changing the data type for :", c)
        df[c] = df[c].astype(np.int64)
        
    int_to_categorical_variables = ['MSSubClass', 'OverallQual', 'OverallCond', 'FireplaceQu', 'MoSold']
    for c in int_to_categorical_variables:
        df[c] = df[c].astype(str)
        
    df = df.fillna('NotAvailable')
    return df
df = giveMeWrangledData(df_concat)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65,8450,Pave,NotAvailable,Reg,Lvl,AllPub,Inside,...,0,NotAvailable,NotAvailable,NotAvailable,0,2,2008,WD,Normal,208500
1,20,RL,80,9600,Pave,NotAvailable,Reg,Lvl,AllPub,FR2,...,0,NotAvailable,NotAvailable,NotAvailable,0,5,2007,WD,Normal,181500
2,60,RL,68,11250,Pave,NotAvailable,IR1,Lvl,AllPub,Inside,...,0,NotAvailable,NotAvailable,NotAvailable,0,9,2008,WD,Normal,223500
3,70,RL,60,9550,Pave,NotAvailable,IR1,Lvl,AllPub,Corner,...,0,NotAvailable,NotAvailable,NotAvailable,0,2,2006,WD,Abnorml,140000
4,60,RL,84,14260,Pave,NotAvailable,IR1,Lvl,AllPub,FR2,...,0,NotAvailable,NotAvailable,NotAvailable,0,12,2008,WD,Normal,250000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 76 columns):
MSSubClass       2919 non-null object
MSZoning         2919 non-null object
LotFrontage      2919 non-null int64
LotArea          2919 non-null int64
Street           2919 non-null object
Alley            2919 non-null object
LotShape         2919 non-null object
LandContour      2919 non-null object
Utilities        2919 non-null object
LotConfig        2919 non-null object
LandSlope        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null object
OverallCond      2919 non-null object
YearBuilt        2919 non-null int64
YearRemodAdd     2919 non-null int64
RoofStyle        2919 non-null object
RoofMatl         2919 non-null object
Exterior1st      2919 non-null object
Exterior2nd      2919 n

In [5]:
def preProcessData(df, log=False):
    
    print("Shape of the data set before pre processing : ", df.shape )

    
    #get dummies
    if log:
        print("Categorical columns : ", list(df.select_dtypes(exclude=np.number)))
    df = pd.get_dummies(df)
    #df = df.drop(categorical_columns, axis=1)
    
    print("\n\nShape of the data set after pre processing : ", df.shape )
    
    if log:
        print("Columns in the data set are : ",list(df))

    return df
df_prep = preProcessData(df)
df_prep.info()

Shape of the data set before pre processing :  (2919, 76)


Shape of the data set after pre processing :  (2919, 351)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Columns: 351 entries, LotFrontage to SaleCondition_Partial
dtypes: int64(29), uint8(322)
memory usage: 1.6 MB


In [6]:
def newBoxCoxTranformation(df,target,testFile=False):
    
    #assuming that only numerical features are presented
    print("Shape of the dataset initial : ", df.shape)
    
    if not testFile:
        df =df[df.SalePrice >0]
        print("Shape of the dataset before transformation : ", df.shape)
        y = np.array(df[target].apply( lambda x: math.log(x)))
        X= df.drop(target,axis = 1)
        #x_columns = list(X)
        X = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X)
        X = preprocessing.power_transform( X, method='box-cox')
        #X = pd.DataFrame(X,columns=x_columns)
        print("Shape of the dataset after transformation : ", X.shape, y.shape)
        return X,y
    else:
        df = df[df.SalePrice == 0.0]
        print("Shape of the dataset before transformation : ", df.shape)
        X=df.drop(target,axis = 1)
        X = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X)
        X = preprocessing.power_transform( X, method='box-cox')
        print("Shape of the dataset after transformation : ", X.shape)
        return X
        
    

    
X,y = newBoxCoxTranformation(df_prep,'SalePrice')

Shape of the dataset initial :  (2919, 351)
Shape of the dataset before transformation :  (1460, 351)


C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: divide by zero encountered in log
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))


Shape of the dataset after transformation :  (1460, 350) (1460,)


C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.5, random_state=random.randint(1,500))#, stratify=df.BldgType)

In [8]:
reg = XGBRegressor()
reg.fit(X_train,y_train)
reg.score(X_test,y_test)

0.8850237590454624

In [9]:
np.sqrt(mean_squared_log_error(y_test, reg.predict(X_test)))

0.010573299185072245

In [10]:
reg.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [11]:
reg

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

##### We need to have different pre-processing logic to test data. We will come back to it little later.

In [18]:
def checkTheTestFile(reg):
    df_test = pd.read_csv('test.csv')
    df_test['SalePrice'] = 0.0
    
    df_train =  pd.read_csv('train.csv')
    df_concat = pd.concat([df_train,df_test])

    #print(df_test[df_test.TotalBsmtSF.isna()])
    #return
    df = giveMeWrangledData(df_concat,True)
    
    #print(df.info())
    df = preProcessData(df)
    #print(df.info())
    X = newBoxCoxTranformation(df,'SalePrice',True)
    #print(np.sqrt(mean_squared_log_error(y, reg.predict(X))))
    
    df_test['SalePrice'] = np.exp(reg.predict(X))
    
    
    return df_test
df_test = checkTheTestFile(reg)

Shape of the data set before pre processing :  (2919, 76)


Shape of the data set after pre processing :  (2919, 351)
Shape of the dataset initial :  (2919, 351)
Shape of the dataset before transformation :  (1459, 351)


C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: divide by zero encountered in log
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))


Shape of the dataset after transformation :  (1459, 350)


C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [19]:
df_test[['Id','SalePrice']]

,Id,SalePrice
0,1461,119739.890625
1,1462,154616.593750
2,1463,189186.187500
3,1464,196393.156250
4,1465,181883.937500
5,1466,184173.125000
6,1467,174647.546875
7,1468,171107.265625
8,1469,197169.156250
9,1470,126812.921875


In [20]:
df_test[['Id','SalePrice']].to_csv('submission.csv',index=False)

##### I got Kaggle Rank of 2539/4463 with RMSLE =0.14357
##### As on 1/17/2019 : 9:06PM IST
* 0.13501 ==> 2040 
* 0.13252 ==> 1865
* 0.13002 ==> 1704
* 0.12658 ==> 1500
* 0.12351 ==> 1250
* 0.12081 ==> 1000
* 0.11572 ==> 500
* 0.11475 ==> 250
* 0.11310 ==> 100
* 0.10985 ==> 50
* 0.10973 ==> 25
* 0.10845 ==> 10
* 0.08021 ==> 5
* 0.00000 ==> 1

##### Now that I know around what score gets what rank; can we have a function which would what would be testing score ?

##### Logic is to predict first the testing samples. Later use that for training and predict the initial training data set. We would then have actual and predicted SalePrices with which we can calculated the RMSLE.

##### Would this logic work ? let us try for our case now and compare that with Kaggle result....Finger crossed :)

In [24]:
def checkTheTestingScore(df_test, max_depth, n_estimator):
 
    df_train =  pd.read_csv('train.csv')
    df_train.SalePrice = 0
    df_concat = pd.concat([df_train,df_test])
    
    X,y = newBoxCoxTranformation(preProcessData((giveMeWrangledData(df_concat))),'SalePrice')
    reg=XGBRegressor(max_depth=max_depth, n_estimator=n_estimator).fit(X,y)
    
    X = newBoxCoxTranformation(preProcessData((giveMeWrangledData(df_concat))),'SalePrice',True)
    df_train =  pd.read_csv('train.csv')
    df_train['predicted_SalePrice']=np.exp(reg.predict(X))
    
    
    print("max_depth :", max_depth, "n_estimator :", n_estimator,\
          "RMSLE : ", np.sqrt(mean_squared_log_error(df_train.SalePrice,df_train.predicted_SalePrice)))
    
    return np.sqrt(mean_squared_log_error(df_train.SalePrice,df_train.predicted_SalePrice))
    
dummy = checkTheTestingScore(df_test,3,100)
print(dummy,3,100)

Shape of the data set before pre processing :  (2919, 76)


Shape of the data set after pre processing :  (2919, 351)
Shape of the dataset initial :  (2919, 351)
Shape of the dataset before transformation :  (1459, 351)


C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: divide by zero encountered in log
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Shape of the dataset after transformation :  (1459, 350) (1459,)
Shape of the data set before pre processing :  (2919, 76)


Shape of the data set after pre processing :  (2919, 351)
Shape of the dataset initial :  (2919, 351)
Shape of the dataset before transformation :  (1460, 351)


C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: divide by zero encountered in log
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))


Shape of the dataset after transformation :  (1460, 350)
max_depth : 3 n_estimator : 100 RMSLE :  0.13242687648293444
0.13242687648293444 3 100


C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


##### This is good news; we have the RMSLE calculation matching amost to that of Kaggle.

In [28]:
score_list = []
for i in range(3,15):
    for j in range(100,900,100):
        reg = XGBRegressor(max_depth=i, n_estimators=j).fit(X,y)
        score_list.append((checkTheTestingScore(checkTheTestFile(reg),i,j),i,j))

Shape of the data set before pre processing :  (2919, 76)


Shape of the data set after pre processing :  (2919, 351)
Shape of the dataset initial :  (2919, 351)
Shape of the dataset before transformation :  (1459, 351)


C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: divide by zero encountered in log
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Shape of the dataset after transformation :  (1459, 350)
Shape of the data set before pre processing :  (2919, 76)


Shape of the data set after pre processing :  (2919, 351)
Shape of the dataset initial :  (2919, 351)
Shape of the dataset before transformation :  (1459, 351)


C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: divide by zero encountered in log
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Shape of the dataset after transformation :  (1459, 350) (1459,)
Shape of the data set before pre processing :  (2919, 76)


Shape of the data set after pre processing :  (2919, 351)
Shape of the dataset initial :  (2919, 351)
Shape of the dataset before transformation :  (1460, 351)


...

C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: divide by zero encountered in log
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Shape of the dataset after transformation :  (1459, 350) (1459,)
Shape of the data set before pre processing :  (2919, 76)


Shape of the data set after pre processing :  (2919, 351)
Shape of the dataset initial :  (2919, 351)
Shape of the dataset before transformation :  (1460, 351)


C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: divide by zero encountered in log
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))


Shape of the dataset after transformation :  (1460, 350)
max_depth : 14 n_estimator : 800 RMSLE :  0.1329451817841083


C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


##### The best score so far is 0.12653 for maxdepth = 4 and n_estimator = 200. Let us see our prediction on Kaggle board right or not.

In [29]:
sorted(score_list,key= lambda x:x[0])

[(0.12371909654413707, 4, 600),
 (0.12433296116361359, 5, 800),
 (0.12448644793947504, 4, 300),
 (0.12484209537059474, 4, 800),
 (0.1252574946869062, 4, 500),
 (0.12535504740406198, 4, 700),
 (0.12537015230697637, 5, 600),
 (0.1253926717494223, 5, 500),
 (0.12540132404374835, 5, 400),
 (0.12567539969077682, 6, 600),
 (0.1260081331304752, 4, 400),
 (0.1261639809280237, 5, 700),
 (0.1262380436951287, 8, 100),
 (0.1263241495879595, 6, 700),
 (0.12643750270564094, 6, 300),
 (0.12649598894260672, 3, 800),
 (0.12653420820826308, 4, 200),
 (0.12671548847931782, 6, 500),
 (0.1269647674201413, 8, 200),
 (0.1271855778090121, 6, 400),
 (0.12731083490924186, 7, 300),
 (0.12734157350900507, 5, 200),
 (0.12737129901973382, 9, 300),
 (0.12739964511334392, 6, 800),
 (0.1274404122758491, 9, 400),
 (0.1274404122758491, 9, 500),
 (0.1274404122758491, 9, 600),
 (0.1274404122758491, 9, 700),
 (0.1274404122758491, 9, 800),
 (0.12750857434835952, 3, 400),
 (0.12756710284148876, 5, 300),
 (0.1280882874902315,